In [2]:
import pandas as pd
import os

In [3]:
RAW_DATA_DIR = "output/results/25"
import os
# The raw data format: run,best_fitness,num_vehicles,runtime
# Output report format: num_vehicles,min_fitness,avg_fitness,std_fitness,avg_runtime,std_runtime | num)_vehicles: is the number of vehicle associtated with the min fitness
# Load all data files in RAW_DATA_DIR, transform, aggregate and save the report

instances = os.listdir(RAW_DATA_DIR)
instances.sort()

report = []
for instance in instances:
    data = pd.read_csv(f"{RAW_DATA_DIR}/{instance}")
    describe = data.describe()
    min_fitness = describe.loc["min", "best_fitness"]
    avg_fitness = describe.loc["mean", "best_fitness"]
    std_fitness = describe.loc["std", "best_fitness"]
    avg_runtime = describe.loc["mean", "runtime"]
    std_runtime = describe.loc["std", "runtime"]
    num_vehicles = data.loc[data["best_fitness"] == min_fitness, "num_vehicles"].values[0]
    report.append([num_vehicles, min_fitness, avg_fitness, std_fitness, avg_runtime, std_runtime])

report = pd.DataFrame(report, columns=["num_vehicles", "min_fitness", "avg_fitness", "std_fitness", "avg_runtime", "std_runtime"])
report.to_csv("output/25_report.csv", index=False)
report.head()

,num_vehicles,min_fitness,avg_fitness,std_fitness,avg_runtime,std_runtime
0,3,165.951197,165.951197,2.995911e-14,3.965793,0.184369
1,2,134.353015,134.479031,3.984957e-01,4.184238,0.131589
2,2,153.546347,162.446770,4.919524e+00,3.988025,0.064466
3,2,160.007720,160.007720,0.000000e+00,4.142815,0.066599
4,2,173.722430,173.722430,2.995911e-14,4.089193,0.078275


In [ ]:
# Extract from .txt
# Read from text file. Text format:
# # Every 1st row: instance name
# # Every 2nd row: benchmark num_vehicles (can be ignored and skipped)
# # Every 3rd row: benchmark runtime (can be ignored and skipped)
# # Every 4th row: proposed num_vehicles
# # Every 5th row: best fitness
# # Every 6th row: avg fitness
# # Every 7th row: runtime
# # Every 8th row: gap 1 (can be ignored and skipped)
# # Every 9th row: gap 2 (can be ignored and skipped)

# The output format: instance_name,num_vehicles,min_fitness,avg_fitness,runtime
# Load data from text.txt, transform, aggregate and save the report

TEXT_FILE = "output/test.txt"
with open(TEXT_FILE, "r") as f:
    lines = f.readlines()

report = []
for i in range(0, 31, 10):
    instance_name = lines[i].strip()
    print(instance_name)
    num_vehicles = int(lines[i + 4].strip())
    min_fitness = float(lines[i + 5].strip())
    avg_fitness = float(lines[i + 6].strip())
    runtime = float(lines[i + 7].strip())
    report.append([instance_name, num_vehicles, min_fitness, avg_fitness, runtime])

report = pd.DataFrame(report, columns=["instance_name", "num_vehicles", "min_fitness", "avg_fitness", "runtime"])
report.to_csv("output/text_report.csv", index=False)
report.head()

C03
C204
C205
C206
C207
C208
R101
R102
R103
11104
R105
R106
R107
R108
R109
R110
R111
RI
0.00%


ValueError: invalid literal for int() with base 10: '653.3'